# Reshaping and Merging `DataFrames` -- Exercises

## Goal

Practice reshaping with `pivot` and `melt`, and merging with `concat` and `merge`

## Exercises

### 0. Import `pandas` and load the gapminder data set

In [ ]:
import pandas as pd

# This is actually a tab separated file (not comma); still use read_csv, but specify the tab separator
gapminder = pd.read_csv("https://raw.githubusercontent.com/jennybc/gapminder/master/inst/extdata/gapminder.tsv", sep="\t")

### 1. Make a wide version of the data giving the population for each year across the countries, with each country as an individual column

In [ ]:
# Each row represents a specific year, and we need to pivot the countries to the columns
gapminder.pivot(index="year", columns="country", values="pop")

### 2a. Create a subset of the gapminder data following these steps:
1. Get data for just Belgium and Uganda
2. Exclude the first 3 rows of this subset
3. Get just the country, year and population columns

Hint: you don't need to do the steps in a single expression; take it step by step
Hint: you can use the slice operator, `start_index:end_index`, to specify a range of rows to select

In [ ]:
# First get just the countries indicated
bu_data = gapminder[gapminder["country"].isin(["Belgium", "Uganda"])]

# Next, make a slice to remove the first 3 rows
bu_data = bu_data[3:bu_data.shape[0]]

# Finally get the indicated columns
# Also reset the row indices so we can easily count the number of rows
bu_data = bu_data[["country", "year", "pop"]].reset_index(drop = True)

bu_data

### 2b. Make a wide version of the data table you created in 2a, with years along the rows, countries along the columns, and population for the values.  Why are there missing values?

In [ ]:
# Pivot like above
bu_data_wide = bu_data.pivot(index="year", columns="country", values="pop").reset_index()
bu_data_wide

# The missing values in the ouput are due to removing the first three rows in the step above.
# We removed data for Belguium for years 1952, 1957 and 1962, however Uganda has data for these
# years.  This creates "holes" in the pivoted data table which get filled in with NaN (missing values).

### 2c. Convert the wide data from 2b back to long format (like the original) and compare the results to those from 2a.  Are they the same or different? Why?  What does this say about pivot and melt being strict inverses of each other?

In [ ]:
# Melt the wide data to get back to long data
bu_data_long = bu_data_wide.melt(id_vars="year")
bu_data_long

# Aside from the different ordering of the columns, the results from 2a are different:
# 2a had 21 rows, while we have 24 rows here.
# When 2a was made wider, missing values were generated in the resulting table because
# we removed the first 3 rows in 2a.  Melting back to long form preserves these missing
# entries resulting in a longer table vs. the 2a (we got the first 3 rows back but with missing data)
#
# This means that pivoting data from long to wide forms, then melting back to long doesn't mean you'll
# necessarily get back the exact same data table you started with.

### 3. Create a new hypothetical country, measurement year, and other data values, and add this to the gapminder data table

In [ ]:
# Need to make sure the same column names are used
hypo_data = {"country": ["Kingdom of the North"], \
             "continent": ["Westeros"], \
             "year": [297], \
             "lifeExp": [42], \
             "pop": [3750000], \
             "gdpPercap": [50]}
hypo_df = pd.DataFrame(hypo_data)

# This adds the above data as a new row in the table
pd.concat([gapminder, hypo_df])

### 4. A `DataFrame` with hemisphere location (north or south) for a few countries is given below.  Consider merging this data into the gapminder data set - how would the output differ if you did a "right" merge versus a "left" merge?  Do the two merges to confirm your guess.

In [ ]:
hemisphere_data = pd.DataFrame({"country": ["United States", "Chile", "India", "New Zealand"], \
                                "hemisphere": ["north", "south", "north", "south"]})
hemisphere_data

In [ ]:
# == First the left merge
# The left merge will keep all of the rows in the first data table given in 
# the merge command; here we used the gapminder data set so we'd expect to
# have a very long output table with all the rows from the gapminder set.
#
# Note that we have a new column called hemisphere in the output, but
# most values are NaN... why?
#
left_merge = pd.merge(gapminder, hemisphere_data, how="left", on="country")
left_merge

In [ ]:
# Subset to the countries present in hemisphere data to make sure
# the data provided is actually in the table
left_merge[left_merge["country"].isin(["United States", "Chile", "India", "New Zealand"])].reset_index(drop=True)

In [ ]:
# == Now the right merge
# In this case, the "right" or second table given in the merge command is
# the hemisphere data from above.  In this case, we expect to only
# get the countries present in the hemisphere data
#
# Note: the right merge acted like a left merge + subset!
right_merge = pd.merge(gapminder, hemisphere_data, how="right", on="country")
right_merge